In [1]:
# import sys
# !{sys.executable} -m pip install beautifulsoup4

from bs4 import BeautifulSoup as bs
import requests as rs
import pandas as pd
import numpy as np

### 1. The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

In [2]:
# Gathering the data

url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
contents = rs.get(url).text

soup = bs(contents)
table = soup.find('table', attrs={'class':'wikitable sortable'})
trs = table.find_all('tr')
cols = ['PostalCode', 'Borough', 'Neighborhood']
rows = []

for tr in trs:
    td = tr.find_all('td')
    row = [ele.text.strip() for ele in td]
    if row:
        rows.append(row)

df = pd.DataFrame(rows, columns=cols)
display(df.head())


,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


### 2. Only process the cells that have an assigned borough. Ignore cells with a borough that is 'Not assigned'.

In [3]:
df = df[df.Borough != 'Not assigned']
df.reset_index(inplace=True, drop=True)
display(df.head())

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


### 3. If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

In [4]:
df['Neighborhood'] = df.apply(lambda row: row['Borough'] if row['Neighborhood'] == 'Not assigned' else row['Neighborhood'], axis=1)
display(df.head())

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


### 4. More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

In [5]:
df = df.groupby(['PostalCode', 'Borough'])['Neighborhood'].apply(', '.join).to_frame()
df.reset_index(inplace=True)
display(df.head())

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


### 5. Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.

The assignment mentions assigning the name of the borough to the neighbourhood after joining the names of the different neighbourhoods in case they are assigned to multiple postal codes but in this case, these steps were flipped. This was done to ensure that each postal code had a neighborhood assigned to it and that combining the names of the different neighbourhoods would not cause 'Not assigned' to be one of the neighborhoods listed. This was important as it would be much more difficult to find the occurrence of a 'Not assigned' and would require reading each entry individually, using regex to find the text, and then figuring out which borough should placed instead of it. Flipping the steps has the same desired result but is done much more easily.

### 6. In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [6]:
print("\n The shape of the final DataFrame is:", df.shape)


 The shape of the final DataFrame is: (103, 3)
